In [3]:
import time
import random
import itertools
import collections
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from otmtd.utils.distance import WTE
from scipy.spatial import distance


class EmbeddingDataset(Dataset):
    def __init__(self, task, text_path, train_embs_path, valid_embs_path=None, pt_label='domain'):
        embs, labels, unique_labels = self.load_embs_and_labels(task, text_path, train_embs_path, valid_embs_path, pt_label)
        if task != 'pre_train':
            print("{} unique labels: {}".format(task.capitalize(), len(unique_labels)))
        else:
            print("PRE-TRAIN {} unique labels: {}".format(pt_label, len(unique_labels)))
        counter = collections.Counter(labels)
        sorted_counter = dict(sorted(counter.items(), key=lambda kv: kv[1], reverse=True))
        print(sorted_counter)
        self.unique_labels = unique_labels

        self.embs_and_labels = pd.concat([embs, labels], axis=1)
        # 创建otdd需求的Dataset属性
        self.classes = [str(k) for k in unique_labels] # list of unique labels string
        self.targets = torch.tensor(list(labels))

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        embedding = self.embs_and_labels.iloc[idx]['pro_emb']
        label = self.embs_and_labels.iloc[idx]['label']
        return torch.tensor(embedding).reshape(1, -1, 1), torch.tensor(label)

    def load_embs_and_labels(self, task, text_path, train_embs_path, valid_embs_path, pt_label):
        texts = pd.read_csv(text_path)
        if task=='pre_train':
            texts.rename(columns={pt_label: 'label'}, inplace=True)
        labels = texts['label']
        unique_labels = labels.unique()

        if task=='pre_train' or task=='kinase': # pre_train / kinase, only `train` set
            embs = pd.read_pickle(train_embs_path)
        else:
            train_embs, valid_embs = pd.read_pickle(train_embs_path), pd.read_pickle(valid_embs_path)
            embs = pd.concat([train_embs, valid_embs], axis=0)

        selected_ids = texts['uniprot_id'].tolist()
        selected_embs_flag = embs['pro_id'].map(lambda x: True if x in selected_ids else False)
        embs = embs[selected_embs_flag].reset_index(drop=True)
        embs = embs['pro_emb']

        return embs, labels, unique_labels
    
def set_random_seed(SEED):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)

def cal_combs_distances_mean(all_pt_tasks, t2t_distances_list):
    mean_t2t_distances = []
    for pt_task in all_pt_tasks:
        spec_pt_task_distance = []
        for t2t_distances in t2t_distances_list:
            if pt_task in t2t_distances['pt_task'].tolist():
                spec_pt_task_distance.append(t2t_distances[t2t_distances['pt_task']==pt_task].to_numpy()[:, :-1].reshape(1, -1))
        spec_pt_task_distance = np.concatenate(spec_pt_task_distance, axis=0).astype(np.float32).mean(axis=0)
        mean_t2t_distances.append(spec_pt_task_distance)
    mean_t2t_distances = np.stack(mean_t2t_distances, axis=0)
    return mean_t2t_distances

In [5]:
return_t2t = True # 同时计算task-to-task distance (for PT and FT)
num_samples = 1000
TaskCombs = ['MLM+RMD', 'GO+RMD', 'MLM+GO+D', 'RMD']
pt_tasks_combs = [
    ['mlm', 'domain', 'motif', 'region'],
    ['domain', 'motif', 'region', 'go'],
    ['mlm', 'domain', 'go'],
    ['domain', 'motif', 'region']
]

device = torch.device('cpu')
# generate reference
feat_dim, lbl_emb_dim, ref_size = 512, 10, 200
seed = 1145114
set_random_seed(seed)
reference = torch.randn(ref_size, feat_dim + lbl_emb_dim, dtype=float, device=device).float()

wte_distance, wte_t2t_distances = [], []
texts_base_dir = "/home/brian/work/OTMTD_GH/processed_data"
embs_base_dir = "/home/brian/work/OTMTD_GH/protein_embeddings_MultiTasks"
for i, comb in enumerate(TaskCombs):
    print("Start computing for Pretrain-MultiTask: {}".format(comb))
    # 根据multi-tasks combination构建对应的embdding hub
    tasks_texts_embs_hub = {
        'pre_train': ["{}/pre_train/sampling_set.txt".format(texts_base_dir),
                        "{}/pre_train_combs/pre_train_{}_pro_embs_pt.pkl".format(embs_base_dir, comb)],
        'stability': ["{}/stability/sequence_go_label_{}.txt".format(texts_base_dir, num_samples),
                        "{}/stability/{}->stability_pretrain_pro_embs_train.pkl".format(embs_base_dir, comb),
                        "{}/stability/{}->stability_pretrain_pro_embs_valid.pkl".format(embs_base_dir, comb)],
        'fluoresecence': ["{}/fluorescence/sequence_go_label_{}.txt".format(texts_base_dir, num_samples),
                        "{}/fluorescence/{}->fluorescence_pretrain_pro_embs_train.pkl".format(embs_base_dir, comb),
                        "{}/fluorescence/{}->fluorescence_pretrain_pro_embs_valid.pkl".format(embs_base_dir, comb)],
        'remote_homology': ["{}/remote_homology/sequence_go_label_{}.txt".format(texts_base_dir, num_samples),
                            "{}/remote_homology/{}->remote_homology_pretrain_pro_embs_train.pkl".format(embs_base_dir, comb),
                            "{}/remote_homology/{}->remote_homology_pretrain_pro_embs_valid.pkl".format(embs_base_dir, comb)], 
        'secondary_structrue': ["{}/secondary_structure/sequence_go_label_{}.txt".format(texts_base_dir, num_samples),
                                "{}/secondary_structure/{}->secondary_structure_pretrain_pro_embs_train.pkl".format(embs_base_dir, comb),
                                "{}/secondary_structure/{}->secondary_structure_pretrain_pro_embs_valid.pkl".format(embs_base_dir, comb)],
        'pdbbind': ["{}/pdbbind/sequence_go_label_{}.txt".format(texts_base_dir, num_samples),
                    "{}/pdbbind/{}->pdbbind_pretrain_pro_embs_train.pkl".format(embs_base_dir, comb),
                    "{}/pdbbind/{}->pdbbind_pretrain_pro_embs_dev.pkl".format(embs_base_dir, comb)],
        'kinase': ["{}/kinase/sequence_go_label_{}.txt".format(texts_base_dir, num_samples),
                    "{}/kinase/{}->kinase_pretrain_pro_embs_train.pkl".format(embs_base_dir, comb)]
    }

    pt_datasets, pt_class_nums, pt_names = [], [], []
    for label_task in pt_tasks_combs[i]:
        dataset = EmbeddingDataset('pre_train', *tasks_texts_embs_hub['pre_train'], pt_label=label_task)
        pt_datasets.append(dataset)
        pt_class_nums.append(len(dataset.unique_labels))
        pt_names.append(label_task)

    ft_datasets, ft_class_nums, ft_names = [], [], []
    for task in ['stability', 'fluoresecence', 'remote_homology', 'secondary_structrue', 'pdbbind', 'kinase']:
        dataset = EmbeddingDataset(task, *tasks_texts_embs_hub[task])
        ft_datasets.append(dataset)
        ft_class_nums.append(len(dataset.unique_labels))
        ft_names.append(task.capitalize())

    pt_tasks_dict = {task: i for i, task in enumerate(pt_tasks_combs[i])}
    MultiTask_WTE = WTE(lbl_emb_dim, device, pt_class_nums=np.array(pt_class_nums),
                        ft_class_nums=np.array(ft_class_nums), pt_names=pt_names, ft_names=ft_names,
                        pt_tasks_combs=[pt_tasks_combs[i]], pt_tasks_dict=pt_tasks_dict, gaussian_assumption=True)
    # pt_task_comb_embs, ft_task_embs = MultiTask_WTE.cwte(pt_datasets, ft_datasets, reference)
    pt_task_comb_embs, ft_task_embs, pt_task_sole_embs = MultiTask_WTE.cwte(pt_datasets, ft_datasets, reference, return_t2t=return_t2t)
    pt_task_comb_vecs = pt_task_comb_embs.reshape(1, -1) # each comb each time
    pt_task_sole_vecs = pt_task_sole_embs.reshape(len(pt_datasets), -1) # (PT dataset_num, ref_size * (feat_dim+lbl_emb_dim))
    ft_task_vecs = ft_task_embs.reshape(ft_task_embs.shape[0], -1) # (dataset_num, ref_size * (feat_dim+lbl_emb_dim)) flatten operation

    wte_distance_row = distance.cdist(pt_task_comb_vecs, ft_task_vecs, 'euclidean') # (dataset_num, dataset_num)
    wte_distance.append(wte_distance_row)
    wte_sole_distance = distance.cdist(pt_task_sole_vecs, ft_task_vecs, 'euclidean')
    wte_sole_distance = pd.DataFrame(wte_sole_distance, columns=ft_names)
    wte_sole_distance['pt_task'] = pt_names
    wte_t2t_distances.append(wte_sole_distance)
    print(wte_distance_row.shape, '\n', '==== x ==== ' * 5)
    
wte_distance = np.concatenate(wte_distance, axis=0)
wte_t2t_distances_ = cal_combs_distances_mean(['mlm', 'domain', 'motif', 'region', 'go'], wte_t2t_distances)
np.save("./results(all)/wte_distance_20bins_{}_mlm.npy".format(num_samples), wte_distance)
np.save("./results(all)/wte_t2t_distances.npy", wte_t2t_distances_)

Start computing for Pretrain-MultiTask: MLM+RMD
PRE-TRAIN mlm unique labels: 15
{2: 148, 10: 117, 6: 116, 8: 101, 9: 98, 4: 95, 5: 59, 14: 54, 1: 49, 3: 47, 7: 46, 12: 45, 0: 45, 11: 13, 13: 12}
PRE-TRAIN domain unique labels: 13
{0: 173, 1: 165, 2: 121, 3: 89, 4: 88, 5: 64, 6: 59, 7: 53, 8: 50, 9: 47, 10: 46, 11: 45, 12: 45}
PRE-TRAIN motif unique labels: 5
{0: 843, 1: 59, 2: 50, 3: 47, 4: 46}
PRE-TRAIN region unique labels: 10
{0: 202, 1: 173, 2: 165, 3: 121, 4: 89, 5: 88, 6: 64, 7: 53, 8: 45, 9: 45}
Stability unique labels: 12
{0: 101, 1: 87, 2: 83, 3: 57, 4: 53, 5: 34, 6: 27, 7: 20, 8: 14, 9: 11, 10: 7, 11: 6}
Fluoresecence unique labels: 12
{0: 142, 1: 83, 2: 80, 3: 48, 4: 38, 5: 35, 6: 20, 7: 20, 8: 14, 9: 9, 10: 6, 11: 5}
Remote_homology unique labels: 16
{0: 136, 1: 126, 2: 96, 3: 26, 4: 20, 5: 14, 6: 11, 7: 11, 8: 9, 9: 9, 10: 8, 11: 8, 12: 7, 13: 7, 14: 6, 15: 6}
Secondary_structrue unique labels: 15
{9: 55, 3: 50, 1: 46, 14: 45, 8: 42, 7: 38, 13: 38, 0: 32, 6: 30, 11: 28, 2:

13it [00:01, 10.57it/s]
Computing label-to-label distances: 100%|██████████| 195/195 [00:21<00:00,  8.98it/s]


Computing inter label-to-label distance for mlm & motif


5it [00:00, 17.21it/s]
Computing label-to-label distances: 100%|██████████| 75/75 [00:06<00:00, 11.26it/s]


Computing inter label-to-label distance for mlm & region


...

Computing intra label-to-label distance for Pdbbind


11it [00:00, 20.92it/s]
Computing label-to-label distances: 100%|██████████| 55/55 [00:02<00:00, 18.55it/s]


Computing intra label-to-label distance for Kinase


2it [00:00, 16.27it/s]
Computing label-to-label distances: 100%|██████████| 1/1 [00:00<00:00, 18.33it/s]


Finish label embedding in 4.5 mins
Wasserstein embedding...
Finish WTE in 4.5 mins
(1, 6) 
 ==== x ==== ==== x ==== ==== x ==== ==== x ==== ==== x ==== 


In [6]:
wte_distance

array([[38.40540346, 44.79597617, 33.85493267, 28.63922014, 24.94117958,
        32.10071586],
       [46.82312473, 48.46425588, 40.7069211 , 36.05075889, 31.85468649,
        37.19391109],
       [46.34556328, 49.16766106, 41.38405414, 33.45248207, 30.8719547 ,
        36.89193618],
       [45.22778648, 48.96813173, 41.61017046, 36.56553857, 32.70094148,
        36.49767181]])

In [7]:
wte_t2t_distances_

array([[61.670288, 63.41589 , 56.41153 , 50.012352, 45.240467, 49.394222],
       [56.66255 , 57.94836 , 52.118855, 46.64529 , 42.66111 , 46.740273],
       [47.307526, 52.377655, 44.003674, 41.161655, 38.047565, 42.479202],
       [44.199158, 48.80211 , 40.563343, 36.305386, 35.674587, 40.402863],
       [40.889996, 47.152973, 37.526886, 34.33467 , 35.356403, 40.64837 ]],
      dtype=float32)